In [ ]:
!conda install -c conda-forge gdcm -y

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import cv2
import matplotlib.pyplot as plt
import pydicom as dicom
from pydicom.data import get_testdata_files

import os
from tqdm import tqdm
import glob

In [ ]:
def load_image(path):
    ds=dicom.dcmread(path)
    img = ds.pixel_array                                  # Now, img is pixel_array. it is input of our demo code
                                                          # Convert pixel_array (img) to -> gray image (img_2d_scaled)
    img_2d = img.astype(float)                            # Step 1. Convert to float to avoid overflow or underflow losses.
    img = (np.maximum(img_2d,0) / img_2d.max()) * 255.0   # Step 2. Rescaling grey scale between 0-255
    return img

In [ ]:
def load_image(path):
    ds=dicom.dcmread(path)
    img = ds.pixel_array                                  # Now, img is pixel_array. it is input of our demo code
                                                          # Convert pixel_array (img) to -> gray image (img_2d_scaled)
    img_2d = img.astype(float)                            # Step 1. Convert to float to avoid overflow or underflow losses.
    img_2d = img_2d - img_2d.min()
    img = (img_2d / img_2d.max()) * 255.0   # Step 2. Rescaling grey scale between 0-255
    return img

In [ ]:
# 胸を楕円と仮定.下記ラマヌジャンの近似式により胸囲を計算.
# https://www.mathsisfun.com/geometry/ellipse-perimeter.html
# - Approximation 2
def chest_circumference(a, b):
    return np.pi*(3*(a + b) - np.sqrt((3*a+b) * (a+3*b)))

In [ ]:
# f, ax= plt.subplots(figsize=(6,6))
columns = ['PatientID', 'Chest']
chest_df = pd.DataFrame(index=[], columns=columns)

path='../input/osic-pulmonary-fibrosis-progression/train/'

img_list = []
for patient in os.listdir(path):
    path='../input/osic-pulmonary-fibrosis-progression/train/'
    path = path + patient
    # 胸のCT画像取得のため中間地点のdcmを取得
    dcm_files = sorted(glob.glob(path+'/*.dcm'))
    glob.glob(path+'/*.dcm')
    filename = []
    for dcm_file in dcm_files:
        file = os.path.splitext(os.path.basename(dcm_file))[0]
        filename.append(int(file))
    chest_loc = int(max(filename) / 2)
    img_path = os.path.join(path, str(chest_loc))
    img_path = img_path + '.dcm' # このパスの画像に対して胸囲を計算する
    ds = dicom.dcmread(img_path)
    tag_list = ['PixelSpacing']
    pixel_spacing = ds.PixelSpacing
    rows = ds.Rows
    cols = ds.Columns
    img = ds.pixel_array                                

    img = load_image(img_path)
    ret2,img_thr = cv2.threshold(img.astype('uint8'), 0, 255, cv2.THRESH_OTSU)
    #ret2,img_thr = cv2.threshold(img.astype('uint8'), 0, 255, cv2.THRESH_TRIANGLE)
    #img_thr = cv2.threshold(img.astype('uint8'), 0, 255, 0)[1]
    #img_d = np.where(img<140,0,img)
    #img_r = np.where(img_d>170,0,img_d)
    #img_thr = np.where(img_r<1,0,255)
    
    # CT画像全体の半分の位置のたて、横の値を取得
    width = img_thr[int(rows / 2),:]
    height = img_thr[:, int(cols / 2)]

    width_white = np.where(width==255)[0] # CT画像の横の中心部分を抽出
    height_white = np.where(height==255)[0]
    
    
    if (len(width_white) != 0) & (len(height_white) != 0):
        chest_half_height = (max(height_white) - min(height_white)) / 2
        chest_half_height = chest_half_height * pixel_spacing[1] / 10
        chest_half_width = (max(width_white) - min(width_white)) / 2 # 胸部の横の長さの半分を取得
        chest_half_width = chest_half_width * pixel_spacing[0] / 10 # convert pixel to cm

        chest = chest_circumference(chest_half_width, chest_half_height)    
        chest_df = chest_df.append({'PatientID': patient, 'Chest': chest}, ignore_index=True)
    else:
        print(patient)
        #print(width)
        #print(np.unique(img_thr))
        #print(np.unique(img))
        #print(ret2)
        #plt.imshow(img_thr)
        #print(width)

In [ ]:
chest_df.to_csv("chest_df.csv")